## 准备数据

In [59]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [60]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [61]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        num = 1000
        self.W1 = tf.Variable(shape=[28 * 28, num], dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[28 * 28, num],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[num], dtype=tf.float32, initial_value=tf.zeros(num))
        self.W2 = tf.Variable(shape=[num, 10], dtype=tf.float32,
                                initial_value=tf.random.uniform(shape=[num, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        flat_x = tf.reshape(x, shape=[-1, 28 * 28])
        h1 = tf.nn.relu(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [62]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [63]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                    tf.constant(test_data[0], dtype=tf.float32), 
                    tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3905141 ; accuracy 0.20203333
epoch 1 : loss 2.337947 ; accuracy 0.21081667
epoch 2 : loss 2.291935 ; accuracy 0.21888334
epoch 3 : loss 2.2509954 ; accuracy 0.22891666
epoch 4 : loss 2.214041 ; accuracy 0.23996666
epoch 5 : loss 2.1802578 ; accuracy 0.25273332
epoch 6 : loss 2.1490245 ; accuracy 0.26703334
epoch 7 : loss 2.119862 ; accuracy 0.28203332
epoch 8 : loss 2.0923965 ; accuracy 0.2974
epoch 9 : loss 2.0663376 ; accuracy 0.31305
epoch 10 : loss 2.0414567 ; accuracy 0.33028334
epoch 11 : loss 2.017575 ; accuracy 0.34721667
epoch 12 : loss 1.9945508 ; accuracy 0.36391667
epoch 13 : loss 1.9722735 ; accuracy 0.38
epoch 14 : loss 1.9506553 ; accuracy 0.39553332
epoch 15 : loss 1.9296277 ; accuracy 0.4103
epoch 16 : loss 1.9091341 ; accuracy 0.4246
epoch 17 : loss 1.8891299 ; accuracy 0.43905
epoch 18 : loss 1.8695794 ; accuracy 0.45206666
epoch 19 : loss 1.8504533 ; accuracy 0.4649
epoch 20 : loss 1.8317275 ; accuracy 0.47716665
epoch 21 : loss 1.8133824 ; accurac